In [ ]:
#conda activate autism-tf
# !pip install tensorflow
# !python -m pip install --upgrade pip setuptools wheel numpy
#!python -m pip install --upgrade --force-reinstall tensorflow
# !pip install matplotlib
# !pip install Pillow
# !pip install scipy
# !pip install pandas
!pip install opencv-python




In [1]:
from tensorflow.keras.applications import ResNet50, VGG16
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
print("Import successful!")


Import successful!


In [ ]:


# Load pretrained ResNet50 and VGG16 models
resnet_model = ResNet50(weights='imagenet')
vgg16_model = VGG16(weights='imagenet')

print("Models loaded successfully!")


In [ ]:


def show_top_images(folder_path, num_images=10):
    # For folders without subfolders (like train)
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_files = image_files[:num_images]

    print(f"Showing images from {os.path.basename(folder_path)}:")
    plt.figure(figsize=(15, 3))
    for i, img_file in enumerate(image_files):
        img_path = os.path.join(folder_path, img_file)
        img = mpimg.imread(img_path)
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(img_file)
    plt.show()

def show_top_images_in_subfolders(parent_folder, num_images=10):
    # For folders with subfolders (like valid)
    subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]
    for subfolder in subfolders:
        image_files = [f for f in os.listdir(subfolder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_files = image_files[:num_images]

        print(f"Showing images from {os.path.basename(subfolder)} in {os.path.basename(parent_folder)}:")
        plt.figure(figsize=(15, 3))
        for i, img_file in enumerate(image_files):
            img_path = os.path.join(subfolder, img_file)
            img = mpimg.imread(img_path)
            plt.subplot(1, num_images, i+1)
            plt.imshow(img)
            plt.axis('off')
            plt.title(img_file)
        plt.show()

# Adjust paths if needed
train_path = './train'
valid_path = './valid'

print("Train images:")
show_top_images(train_path)

print("Valid images:")
show_top_images_in_subfolders(valid_path)


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# --- Parameters ---
img_size = (224, 224)
batch_size = 32

train_dir = './train'  # flat folder, images directly inside
valid_dir = './valid'  # contains subfolders 'Autistic', 'Non_Autistic'
test_dir = './test'    # flat folder, images directly inside

# --- Data Generators ---

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False  # Important to keep order for evaluation
)

test_filenames = os.listdir(test_dir)
test_df = pd.DataFrame({'filename': test_filenames})

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=None,
    target_size=img_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# --- Build Hybrid Model ---

resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=img_size + (3,))
vgg_base = VGG16(include_top=False, weights='imagenet', input_shape=img_size + (3,))

resnet_base.trainable = False
vgg_base.trainable = False

input_layer = Input(shape=img_size + (3,))
resnet_features = resnet_base(input_layer)
vgg_features = vgg_base(input_layer)

resnet_gap = GlobalAveragePooling2D()(resnet_features)
vgg_gap = GlobalAveragePooling2D()(vgg_features)

combined = concatenate([resnet_gap, vgg_gap])
x = Dense(256, activation='relu')(combined)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)

model.compile(optimizer=Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- Train ---

history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10
)

# --- Evaluate on Validation set and show correct/wrong predictions ---

valid_generator.reset()
val_preds_prob = model.predict(valid_generator, verbose=1)
val_preds = (val_preds_prob > 0.5).astype(int).flatten()
val_true = valid_generator.classes
val_filenames = valid_generator.filenames
class_names = list(valid_generator.class_indices.keys())

correct_indices = np.where(val_preds == val_true)[0]
wrong_indices = np.where(val_preds != val_true)[0]

print(f"Total validation samples: {len(val_true)}")
print(f"Correct predictions: {len(correct_indices)}")
print(f"Wrong predictions: {len(wrong_indices)}")

def plot_images(indices, title, max_images=10):
    plt.figure(figsize=(20, 4))
    for i, idx in enumerate(indices[:max_images]):
        img_path = os.path.join(valid_dir, val_filenames[idx])
        img = plt.imread(img_path)
        plt.subplot(1, max_images, i+1)
        plt.imshow(img)
        plt.title(f"Pred: {class_names[val_preds[idx]]}\nTrue: {class_names[val_true[idx]]}")
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

plot_images(correct_indices, "Correct Predictions")
plot_images(wrong_indices, "Wrong Predictions")

# --- Predict on Test set ---

test_generator.reset()
pred_probs = model.predict(test_generator, verbose=1)
pred_labels = (pred_probs > 0.5).astype(int).flatten()

results = list(zip(test_generator.filenames, pred_labels))

print("\nTest Predictions:")
for fname, label in results[:10]:  # first 10 predictions
    class_name = 'Autistic' if label == 0 else 'Non_Autistic'
    print(f"{fname}: {class_name}")


In [ ]:
# test code 2 

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# PARAMETERS
img_size = (280, 280)
batch_size = 32

train_dir = './train'   # Flat folder with labeled filenames (e.g. "Autistic_001.jpg")
valid_dir = './valid'   # Has subfolders: 'Autistic', 'Non_Autistic'
test_dir = './test'     # Flat folder, no labels

# Helper: get label from filename prefix
def get_label(fname):
    if fname.lower().startswith('autistic'):
        return 0
    elif fname.lower().startswith('non_autistic'):
        return 1
    else:
        return None  # Unknown label, skip

# Prepare training filenames and labels
train_filenames = []
train_labels = []

for fname in os.listdir(train_dir):
    label = get_label(fname)
    if label is not None:
        train_filenames.append(fname)
        train_labels.append(label)

# Face detection for cropping
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_crop_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    if len(faces) == 0:
        return None
    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
    (x, y, w, h) = faces[0]
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, img_size)
    return cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

def train_generator(filenames, labels, directory, datagen, batch_size, shuffle=True):
    n = len(filenames)
    indices = np.arange(n)
    while True:
        if shuffle:
            np.random.shuffle(indices)
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_idx = indices[start:end]
            batch_imgs = []
            batch_lbls = []
            for i in batch_idx:
                img_path = os.path.join(directory, filenames[i])
                face_img = detect_and_crop_face(img_path)
                if face_img is None:
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
                    face_img = np.array(img)
                batch_imgs.append(face_img)
                batch_lbls.append(labels[i])
            batch_imgs = np.array(batch_imgs, dtype=np.float32)
            batch_imgs = datagen.standardize(batch_imgs)
            yield batch_imgs, np.array(batch_lbls)

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='reflect'
)

valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Generators
train_gen = train_generator(train_filenames, train_labels, train_dir, train_datagen, batch_size)
valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_filenames = os.listdir(test_dir)
test_df = pd.DataFrame({'filename': test_filenames})
test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=None,
    target_size=img_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

train_steps = len(train_filenames) // batch_size
valid_steps = valid_gen.samples // batch_size

# Build ensemble model
input_layer = Input(shape=img_size + (3,))

resnet_base = ResNet50(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in resnet_base.layers[:-50]:
    layer.trainable = False
resnet_gap = GlobalAveragePooling2D()(resnet_base.output)

vgg_base = VGG16(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in vgg_base.layers[:-50]:
    layer.trainable = False
vgg_gap = GlobalAveragePooling2D()(vgg_base.output)

effnet_base = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in effnet_base.layers[:-50]:
    layer.trainable = False
effnet_gap = GlobalAveragePooling2D()(effnet_base.output)

combined = concatenate([resnet_gap, vgg_gap, effnet_gap])
x = Dense(512, activation='relu')(combined)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=BinaryCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

# Train the model
history = model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=valid_gen,
    validation_steps=valid_steps,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)

# Plot training metrics
def plot_training(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy over epochs')

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss over epochs')

    plt.show()

plot_training(history)

# Validation evaluation and visualization
val_preds_prob = model.predict(valid_gen, steps=valid_steps, verbose=1)
val_preds = (val_preds_prob > 0.5).astype(int).flatten()
val_true = valid_gen.classes
val_filenames = valid_gen.filenames
class_names = list(valid_gen.class_indices.keys())

correct_idx = np.where(val_preds == val_true)[0]
wrong_idx = np.where(val_preds != val_true)[0]

print(f"Validation samples: {len(val_true)}")
print(f"Correct predictions: {len(correct_idx)}")
print(f"Wrong predictions: {len(wrong_idx)}")

def plot_examples(indices, title, max_imgs=10):
    plt.figure(figsize=(20,4))
    for i, idx in enumerate(indices[:max_imgs]):
        img_path = os.path.join(valid_dir, val_filenames[idx])
        img = plt.imread(img_path)
        plt.subplot(1, max_imgs, i+1)
        plt.imshow(img)
        plt.title(f"Pred: {class_names[val_preds[idx]]}\nTrue: {class_names[val_true[idx]]}")
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

plot_examples(correct_idx, "Correct Predictions")
plot_examples(wrong_idx, "Wrong Predictions")

# Test predictions on test folder
test_gen.reset()
test_preds_prob = model.predict(test_gen, verbose=1)
test_preds = (test_preds_prob > 0.5).astype(int).flatten()

print("\nTest Predictions:")
for fname, pred in zip(test_gen.filenames, test_preds):
    label = 'Autistic' if pred == 0 else 'Non_Autistic'
    print(f"{fname}: {label}")

# Predict a single image (useful for random images)
def predict_image(img_path):
    img = detect_and_crop_face(img_path)
    if img is None:
        print("No face detected, resizing full image")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
    x = np.expand_dims(img, axis=0)
    x = resnet_preprocess_input(x.astype(np.float32))
    pred_prob = model.predict(x)[0][0]
    pred_label = 'Autistic' if pred_prob < 0.5 else 'Non_Autistic'
    print(f"Prediction: {pred_label} (Confidence: {pred_prob:.3f})")
    plt.imshow(img)
    plt.title(f"Predicted: {pred_label}")
    plt.axis('off')
    plt.show()

# Example usage:
# predict_image('path/to/random_image.jpg')


In [3]:
from ctypes import cdll
cdll.LoadLibrary('cudnn64_8.dll')
print("cuDNN DLL loaded successfully.")

FileNotFoundError: Could not find module 'cudnn64_8.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [6]:
# train with gpu 
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# --- GPU Setup ---
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPU devices:", tf.config.list_physical_devices('GPU'))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPUs")
    except RuntimeError as e:
        print(e)

# --- PARAMETERS ---
img_size = (280, 280)
batch_size = 32

train_dir = './train'   # Flat folder with labeled filenames (e.g. "Autistic_001.jpg")
valid_dir = './valid'   # Has subfolders: 'Autistic', 'Non_Autistic'
test_dir = './test'     # Flat folder, no labels

# --- Helper: get label from filename prefix ---
def get_label(fname):
    if fname.lower().startswith('autistic'):
        return 0
    elif fname.lower().startswith('non_autistic'):
        return 1
    else:
        return None  # Unknown label, skip

# --- Prepare training filenames and labels ---
train_filenames = []
train_labels = []

for fname in os.listdir(train_dir):
    label = get_label(fname)
    if label is not None:
        train_filenames.append(fname)
        train_labels.append(label)

# --- Face detection for cropping ---
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_crop_face(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    if len(faces) == 0:
        return None
    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
    (x, y, w, h) = faces[0]
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, img_size)
    return cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

# --- Custom train data generator with face crop ---
def train_generator(filenames, labels, directory, datagen, batch_size, shuffle=True):
    n = len(filenames)
    indices = np.arange(n)
    while True:
        if shuffle:
            np.random.shuffle(indices)
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_idx = indices[start:end]
            batch_imgs = []
            batch_lbls = []
            for i in batch_idx:
                img_path = os.path.join(directory, filenames[i])
                face_img = detect_and_crop_face(img_path)
                if face_img is None:
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
                    face_img = np.array(img)
                batch_imgs.append(face_img)
                batch_lbls.append(labels[i])
            batch_imgs = np.array(batch_imgs, dtype=np.float32)
            batch_imgs = datagen.standardize(batch_imgs)
            yield batch_imgs, np.array(batch_lbls)

# --- Data augmentation ---
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='reflect'
)

valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# --- Generators ---
train_gen = train_generator(train_filenames, train_labels, train_dir, train_datagen, batch_size)
valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_filenames = os.listdir(test_dir)
test_df = pd.DataFrame({'filename': test_filenames})
test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=None,
    target_size=img_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

train_steps = len(train_filenames) // batch_size
valid_steps = valid_gen.samples // batch_size

# --- Build ensemble model ---
input_layer = Input(shape=img_size + (3,))

resnet_base = ResNet50(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in resnet_base.layers[:-50]:
    layer.trainable = False
resnet_gap = GlobalAveragePooling2D()(resnet_base.output)

vgg_base = VGG16(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in vgg_base.layers[:-50]:
    layer.trainable = False
vgg_gap = GlobalAveragePooling2D()(vgg_base.output)

effnet_base = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_layer)
for layer in effnet_base.layers[:-50]:
    layer.trainable = False
effnet_gap = GlobalAveragePooling2D()(effnet_base.output)

combined = concatenate([resnet_gap, vgg_gap, effnet_gap])
x = Dense(512, activation='relu')(combined)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output)

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss=BinaryCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

# --- Train the model ---
history = model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    validation_data=valid_gen,
    validation_steps=valid_steps,
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    workers=4,
    use_multiprocessing=True,
)

# --- Plot training metrics ---
def plot_training(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy over epochs')

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss over epochs')

    plt.show()

plot_training(history)

# --- Validation evaluation and visualization ---
val_preds_prob = model.predict(valid_gen, steps=valid_steps, verbose=1)
val_preds = (val_preds_prob > 0.5).astype(int).flatten()
val_true = valid_gen.classes
val_filenames = valid_gen.filenames
class_names = list(valid_gen.class_indices.keys())

correct_idx = np.where(val_preds == val_true)[0]
wrong_idx = np.where(val_preds != val_true)[0]

print(f"Validation samples: {len(val_true)}")
print(f"Correct predictions: {len(correct_idx)}")
print(f"Wrong predictions: {len(wrong_idx)}")

def plot_examples(indices, title, max_imgs=10):
    plt.figure(figsize=(20,4))
    for i, idx in enumerate(indices[:max_imgs]):
        img_path = os.path.join(valid_dir, val_filenames[idx])
        img = plt.imread(img_path)
        plt.subplot(1, max_imgs, i+1)
        plt.imshow(img)
        plt.title(f"Pred: {class_names[val_preds[idx]]}\nTrue: {class_names[val_true[idx]]}")
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

plot_examples(correct_idx, "Correct Predictions")
plot_examples(wrong_idx, "Wrong Predictions")

# --- Test predictions on test folder ---
test_gen.reset()
test_preds_prob = model.predict(test_gen, verbose=1)
test_preds = (test_preds_prob > 0.5).astype(int).flatten()

print("\nTest Predictions:")
for fname, pred in zip(test_gen.filenames, test_preds):
    label = 'Autistic' if pred == 0 else 'Non_Autistic'
    print(f"{fname}: {label}")

# --- Predict a single image (useful for random images) ---
def predict_image(img_path):
    img = detect_and_crop_face(img_path)
    if img is None:
        print("No face detected, resizing full image")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)
    x = np.expand_dims(img, axis=0)
    x = resnet_preprocess_input(x.astype(np.float32))
    pred_prob = model.predict(x)[0][0]
    pred_label = 'Autistic' if pred_prob < 0.5 else 'Non_Autistic'
    print(f"Prediction: {pred_label} (Confidence: {pred_prob:.3f})")
    plt.imshow(img)
    plt.title(f"Predicted: {pred_label}")
    plt.axis('off')
    plt.show()

# Example usage:
# predict_image('path/to/random_image.jpg')


Num GPUs Available:  0
GPU devices: []
Found 100 images belonging to 2 classes.
Found 300 validated image filenames.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 280, 280, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling (Rescaling)         │ (None, 280, 280, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 280, 280, 3)       │               7 │ rescaling[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 280, 280, 3)       │               0 │ normalization[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv_pad (ZeroPadding2D) │ (None, 281, 281, 3)       │               0 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_conv (Conv2D)            │ (None, 140, 140, 32)      │             864 │ stem_conv_pad[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_bn (BatchNormalization)  │ (None, 140, 140, 32)      │             128 │ stem_conv[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ stem_activation (Activation)  │ (None, 140, 140, 32)      │               0 │ stem_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_dwconv                │ (None, 140, 140, 32)      │             288 │ stem_activation[0][0]      │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_bn                    │ (None, 140, 140, 32)      │             128 │ block1a_dwconv[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_activation            │ (None, 140, 140, 32)      │               0 │ block1a_bn[0][0]           │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_squeeze            │ (None, 32)                │               0 │ block1a_activation[0][0]   │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reshape (Reshape)  │ (None, 1, 1, 32)          │               0 │ block1a_se_squeeze[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_reduce (Conv2D)    │ (None, 1, 1, 8)           │             264 │ block1a_se_reshape[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1a_se_expand (Conv2D)    │ (None, 1, 1, 32)          │             288 │ block1a_se_reduce[0][0]    │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 44,321,124 (169.07 MB)

 Trainable params: 36,160,209 (137.94 MB)

 Non-trainable params: 8,160,915 (31.13 MB)

TypeError: fit() got an unexpected keyword argument 'workers'